# Motor Imagery Baseline

Using the actual movement of left and right fist, explore the decoding performance under different EEG-only feature transforms. Note, we use Stratified K-Fold validation instead of leave-one-subject out.

In [1]:
import sys
sys.path.insert(0, "../")
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

from libs.dataloaders import motor

from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-154i0no7 because the default path (/home/anp054/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


## Average Power
Explore impact of Average Power features

In [2]:
dataset = motor.MotorDataset(
    y_keys=["real-left-fist", "real-right-fist"],
    x_params={
        "feature": "AvgFreqPower", 
        "window": -1,
        "prestim": 1, 
        "postim": 2, 
    },
    seed=3
)
X = np.array(dataset[:][0]) # S T F
y = np.array(dataset[:][1]) # S

X = np.squeeze(np.transpose(X, (0,2,1)))
print("X shape (S F):", X.shape)
print("Y counts:", np.unique(y, return_counts=True))

100%|██████████| 3106/3106 [00:00<00:00, 4189.52it/s]

X shape (S F): (3106, 64)
Y counts: (array([0, 1]), array([1553, 1553]))


In [3]:
pca = PCA(n_components=30)
lda = LinearDiscriminantAnalysis()
cv = StratifiedKFold(5, shuffle=True, random_state=42)
clf = Pipeline([('PCA', pca), ('LDA', lda)])
scores = cross_val_score(clf, X, y, cv=cv)

print("PCA-LDA accuracy: %0.3f" % np.mean(scores))

PCA-LDA accuracy: 0.509


## Spectral Power Variance
Explore impact of CSP features, a techinque that maximizes the _variance_ betweeen two windows.

In [4]:
x_params={
    "feature": "FreqPower", # CSP requires time axis
    "window": -1,
    "prestim": 1, 
    "postim": 2, 
}
dataset.set_x_transformer(x_params)
X = np.array(dataset[:][0]) # S slice T F
y = np.array(dataset[:][1]) # S
X = np.squeeze(np.transpose(X, (0,2,3,1)))
print("X shape (S F):", X.shape)
print("Y counts:", np.unique(y, return_counts=True))

100%|██████████| 3106/3106 [00:00<00:00, 45012.62it/s]

X shape (S F): (3106, 64, 160)
Y counts: (array([0, 1]), array([1553, 1553]))


In [5]:
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
lda = LinearDiscriminantAnalysis()
cv = StratifiedKFold(5, shuffle=True, random_state=42)
clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, X, y, cv=cv)
print("CSP-LDA accuracy: %0.3f" % np.mean(scores))

Computing rank from data with rank=None
    Using tolerance 5.5e+02 (2.2e-16 eps * 64 dim * 3.9e+16  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.6e+02 (2.2e-16 eps * 64 dim * 3.9e+16  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.6e+02 (2.2e-16 eps * 64 dim * 4e+16  max singular value)
    Estimated rank (mag): 64
    MAG: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.6e+02 (2.2e-16 eps * 64 dim * 3.9e+16  max singular value)
  

## Variance Feature

In [15]:
x_params={
    "feature": "VarFreqPower",
    "window": -1,
    "prestim": 1, 
    "postim": 2, 
}
dataset.set_x_transformer(x_params)

X = np.array(dataset[:][0]) # S T F
y = np.array(dataset[:][1]) # S

X = np.squeeze(np.transpose(X, (0,2,1)))
print("X shape (S F):", X.shape)
print("Y counts:", np.unique(y, return_counts=True))

100%|██████████| 3106/3106 [00:00<00:00, 3488.89it/s]

X shape (S F): (3106, 64)
Y counts: (array([0, 1]), array([1553, 1553]))


In [16]:
pca = PCA(n_components=30)
lda = LinearDiscriminantAnalysis()
cv = StratifiedKFold(5, shuffle=True, random_state=42)
clf = Pipeline([('PCA', pca), ('LDA', lda)])
scores = cross_val_score(clf, X, y, cv=cv)

print("Var PCA-LDA accuracy: %0.3f" % np.mean(scores))

Var PCA-LDA accuracy: 0.546


## VGG Transform
Explore impact of VGG feature transform (mean power feats + spatial projection)

In [6]:
import torch
def vgg16_augment(model):
    model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-3])

In [7]:
x_params = {
    "feature": "TopomapNN", 
    "prestim": 0.5, 
    "postim": 1.5, 
    "window": -1,
    "stride": -1,
    "model": "vgg16",
    "model_params": {
        "weights": "DEFAULT",
    },
    "model_augment_fn": vgg16_augment,
}
dataset.set_x_transformer(x_params)

X = np.array(dataset[:][0]) # S T F
y = np.array(dataset[:][1]) # S

X = np.squeeze(np.transpose(X, (0,2,1)))
print("X shape (S F):", X.shape)
print("Y counts:", np.unique(y, return_counts=True))

100%|██████████| 3106/3106 [13:28<00:00,  3.84it/s]

X shape (S F): (3106, 4096)
Y counts: (array([0, 1]), array([1553, 1553]))


In [8]:
pca = PCA(n_components=30)
lda = LinearDiscriminantAnalysis()
cv = StratifiedKFold(5, shuffle=True, random_state=42)
clf = Pipeline([('PCA', pca), ('LDA', lda)])
scores = cross_val_score(clf, X, y, cv=cv)

print("VGG PCA-LDA accuracy: %0.3f" % np.mean(scores))

VGG PCA-LDA accuracy: 0.512


In [9]:
x_params = {
    "feature": "TopomapNN", 
    "prestim": 0.5, 
    "postim": 1.5, 
    "window": -1,
    "stride": -1,
    "model": "alexnet",
    "model_params": {
        "weights": "DEFAULT",
    },
    "model_augment_fn": vgg16_augment, # using same transform for alexnet
}
dataset.set_x_transformer(x_params)

X = np.array(dataset[:][0]) # S T F
y = np.array(dataset[:][1]) # S

X = np.squeeze(np.transpose(X, (0,2,1)))
print("X shape (S F):", X.shape)
print("Y counts:", np.unique(y, return_counts=True))

100%|██████████| 3106/3106 [08:12<00:00,  6.30it/s]

X shape (S F): (3106, 4096)
Y counts: (array([0, 1]), array([1553, 1553]))


In [10]:
pca = PCA(n_components=30)
lda = LinearDiscriminantAnalysis()
cv = StratifiedKFold(5, shuffle=True, random_state=42)
clf = Pipeline([('PCA', pca), ('LDA', lda)])
scores = cross_val_score(clf, X, y, cv=cv)

print("AlexNet PCA-LDA accuracy: %0.3f" % np.mean(scores))

AlexNet PCA-LDA accuracy: 0.490


In [11]:
x_params = {
    "feature": "TopomapNN", 
    "prestim": 0.5, 
    "postim": 1.5, 
    "window": -1,
    "stride": -1,
    "model": "efficientnet_v2_l",
    "model_params": {
        "weights": "DEFAULT",
    },
    "model_augment_fn": vgg16_augment, # using same transform for efficientnet
}
dataset.set_x_transformer(x_params)

X = np.array(dataset[:][0]) # S T F
y = np.array(dataset[:][1]) # S

X = np.squeeze(np.transpose(X, (0,2,1)))
print("X shape (S F):", X.shape)
print("Y counts:", np.unique(y, return_counts=True))

100%|██████████| 3106/3106 [20:35<00:00,  2.51it/s] 

X shape (S F): (3106, 1280)
Y counts: (array([0, 1]), array([1553, 1553]))


In [12]:
pca = PCA(n_components=30)
lda = LinearDiscriminantAnalysis()
cv = StratifiedKFold(5, shuffle=True, random_state=42)
clf = Pipeline([('PCA', pca), ('LDA', lda)])
scores = cross_val_score(clf, X, y, cv=cv)

print("EfficientNet PCA-LDA accuracy: %0.3f" % np.mean(scores))

EfficientNet PCA-LDA accuracy: 0.502
